In [1]:
import json
import os
import sys
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

sys.path.append("../../")
import didgelab.calc.fft
from didgelab.calc.sim.cadsd import CADSD
from didgelab.app import get_app
from didgelab.initializer import init_console_no_output
from didgelab.calc.geo import Geo
init_console_no_output()

da_path = "/Users/jane03/workspaces/music/didge/didge-archive"
didge_archive = json.load(open(os.path.join(da_path, "didge-archive.json")))
didge_archive = list(filter(lambda x:x["shape"] == "straight", didge_archive))


In [2]:
get_app().get_config()["sim.correction"] = "none"
get_app().get_config()["sim.grid"] = "even"
get_app().get_config()["sim.grid_size"] = 0.1
get_app().get_config()

{'sim.grid': 'even',
 'sim.correction': 'none',
 'sim.grid_size': 0.1,
 'sim.fmin': 30,
 'sim.fmax': 1000,
 'log_level': 'info'}

In [4]:
didge = didge_archive[0]

geo = Geo(json.load(open(os.path.join(da_path, didge["geometry"]))))
cadsd = CADSD(geo)
freq, impedance = cadsd.compute_impedance()
impedance = np.array(impedance)
ground = cadsd.compute_ground_spektrum(freq, impedance, np.min(freq), np.max(freq))

IndexError: arrays used as indices must be of integer (or boolean) type